asignar a cada cliente el producto más popular que no tenga contratado.

tomar todas xtrain todo de forma general,
y sacar las probabilidades

ojo analizar si la empresa le esta llendo viendo o mal

In [1]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.float_format = '{:,.3f}'.format
pd.set_option('display.max_columns', 100)

import warnings
warnings.filterwarnings('ignore')

/Users/claudiacastro/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
# cargando el dataset con todas las probabilidades de compra de cada cliente
df_probab_compra = pd.read_parquet('df_proba_compra.parquet')
df_probab_compra.head()

,prob_compra
pk_cid,
15891,0.465
16063,0.697
16203,0.837
16502,0.811
17457,0.709


In [13]:
df_socio = pd.read_parquet("https://easy-money-project-bucket.s3.eu-west-3.amazonaws.com/sociodemographic_df.parquet")
df_socio.head()

,Unnamed: 0,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary
0,0,1375586,2018-01-28,ES,29.000,H,35,N,"87,218.100"
1,1,1050611,2018-01-28,ES,13.000,V,23,N,"35,548.740"
2,2,1050612,2018-01-28,ES,13.000,V,23,N,"122,179.110"
3,3,1050613,2018-01-28,ES,50.000,H,22,N,"119,775.540"
4,4,1050614,2018-01-28,ES,50.000,V,23,N,NaN


In [10]:
#df_full[df_full["deceased"] == "S"].groupby("pk_cid").last()

,Unnamed: 0,pk_partition,country_id,region_code,gender,age,deceased,salary
pk_cid,,,,,,,,
81958,13056748,2019-05-28,ES,9.000,V,95,S,"172,531.890"
281352,13128295,2019-05-28,ES,8.000,H,93,S,"376,117.710"
383443,12153225,2019-04-28,ES,41.000,V,43,S,"74,948.130"
396977,13087686,2019-05-28,ES,9.000,H,53,S,"80,128.140"
676170,12726598,2019-05-28,ES,18.000,V,66,S,"133,034.880"
...,...,...,...,...,...,...,...,...
1406450,13224979,2019-05-28,ES,28.000,V,59,S,"46,289.940"
1439465,13368157,2019-05-28,ES,47.000,V,68,S,NaN
1441140,13368042,2019-05-28,ES,14.000,H,60,S,"88,483.500"


In [14]:
df_prod = pd.read_parquet("https://easy-money-project-bucket.s3.eu-west-3.amazonaws.com/products_df.parquet")
df_prod.head()

,Unnamed: 0,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,0,1375586,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
1,1,1050611,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
2,2,1050612,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
3,3,1050613,2018-01-28,1,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0
4,4,1050614,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1


In [18]:
df_full = pd.merge(df_prod, df_socio, on=["pk_cid","pk_partition"], how="left")
df_full.head() 

,Unnamed: 0_x,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,Unnamed: 0_y,country_id,region_code,gender,age,deceased,salary
0,0,1375586,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,0,ES,29.000,H,35,N,"87,218.100"
1,1,1050611,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,1,ES,13.000,V,23,N,"35,548.740"
2,2,1050612,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,2,ES,13.000,V,23,N,"122,179.110"
3,3,1050613,2018-01-28,1,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,3,ES,50.000,H,22,N,"119,775.540"
4,4,1050614,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,4,ES,50.000,V,23,N,NaN


Para este analisis utilizaremos la partición más reciente:

- La partición más reciente contiene los datos más actuales sobre el comportamiento, preferencias y transacciones de los clientes, por tanto las recomendaciones serán más relevantes y precisas.

- La probabilidad de que las recomendaciones sean bien recibidas y que los clientes estén receptivos a ellas sería mayor.


In [19]:
df_last_partition = df_full[df_full['pk_partition']=='2019-05-28']
df_last_partition

,Unnamed: 0_x,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,Unnamed: 0_y,country_id,region_code,gender,age,deceased,salary
5519929,12715896,657826,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,12715896,ES,25.000,H,44,N,"54,493.380"
5519930,12715899,657817,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,12715899,ES,8.000,V,32,N,NaN
5519931,12715982,657986,2019-05-28,0,0,0,0,0,0,0,1,1.000,1.000,1,1,1,0,0,12715982,ES,41.000,H,39,N,"100,993.170"
5519932,12716026,657905,2019-05-28,0,0,0,0,0,1,0,0,0.000,0.000,0,0,0,0,1,12716026,ES,28.000,H,85,N,"154,059.090"
5519933,12716082,657336,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,12716082,ES,28.000,V,38,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,13647304,1166765,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,13647304,ES,50.000,V,22,N,"43,912.170"
5962920,13647305,1166764,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,13647305,ES,26.000,V,23,N,"23,334.990"
5962921,13647306,1166763,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,13647306,ES,50.000,H,47,N,NaN
5962922,13647307,1166789,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,13647307,ES,50.000,H,22,N,"199,592.820"


Eliminamos todos los clientes que aparecen en deceased como S

In [22]:
# se han eliminado los clientes fallecidos, 86 en total
df_last_partition = df_last_partition[df_last_partition["deceased"] != "S"]
df_last_partition

,Unnamed: 0_x,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,Unnamed: 0_y,country_id,region_code,gender,age,deceased,salary
5519929,12715896,657826,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,12715896,ES,25.000,H,44,N,"54,493.380"
5519930,12715899,657817,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,12715899,ES,8.000,V,32,N,NaN
5519931,12715982,657986,2019-05-28,0,0,0,0,0,0,0,1,1.000,1.000,1,1,1,0,0,12715982,ES,41.000,H,39,N,"100,993.170"
5519932,12716026,657905,2019-05-28,0,0,0,0,0,1,0,0,0.000,0.000,0,0,0,0,1,12716026,ES,28.000,H,85,N,"154,059.090"
5519933,12716082,657336,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,12716082,ES,28.000,V,38,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,13647304,1166765,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,13647304,ES,50.000,V,22,N,"43,912.170"
5962920,13647305,1166764,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,13647305,ES,26.000,V,23,N,"23,334.990"
5962921,13647306,1166763,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,13647306,ES,50.000,H,47,N,NaN
5962922,13647307,1166789,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,13647307,ES,50.000,H,22,N,"199,592.820"


Ahora nos quedamos con las columnas que nos interesan para el analisis

In [23]:
columnas_relev = ['pk_cid','short_term_deposit', 'loans', 'mortgage', 'funds', 'securities', 
                  'long_term_deposit',"em_account_pp",	"credit_card"	,"payroll"	,"pension_plan",
                  "payroll_account"	,"emc_account",	"debit_card",	"em_account_p"	,"em_acount"]

df_last_partition = df_last_partition[columnas_relev]
df_last_partition

,pk_cid,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
5519929,657826,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
5519930,657817,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0
5519931,657986,0,0,0,0,0,0,0,1,1.000,1.000,1,1,1,0,0
5519932,657905,0,0,0,0,0,1,0,0,0.000,0.000,0,0,0,0,1
5519933,657336,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,1166765,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
5962920,1166764,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
5962921,1166763,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
5962922,1166789,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1


A continuacion se crea un ranking de productos para recomendar, basándonos en los que el cliente no tiene y ordenándolos según la probabilidad de compra. Esta estrategia permite personalizar las recomendaciones, mejorar la eficiencia de las campañas de marketing y aumentar la satisfacción y fidelización de los clientes.


In [35]:
# 1. Identificar las columnas de productos (todas excepto 'pk_cid')
product_cols = df_last_partition.columns.drop('pk_cid')

# 2. Sumar los valores de cada columna de productos
product_totals = df_last_partition[product_cols].sum()

# 3. Crear un DataFrame con los totales
product_ranking = product_totals.reset_index()
product_ranking.columns = ['Producto', 'Total']

# 4. Ordenar los productos por total en orden descendente
product_ranking = product_ranking.sort_values(by='Total', ascending=False)
product_ranking

,Producto,Total
14,em_acount,"296,334.000"
12,debit_card,"43,247.000"
10,payroll_account,"26,521.000"
11,emc_account,"24,733.000"
9,pension_plan,"17,353.000"
8,payroll,"16,333.000"
5,long_term_deposit,"6,119.000"
7,credit_card,"4,801.000"
4,securities,"1,786.000"
3,funds,"1,315.000"


In [37]:
product_ranking["ranking"] = product_ranking["Total"].rank(ascending=False)
product_ranking

,Producto,Total,ranking
14,em_acount,"296,334.000",1.000
12,debit_card,"43,247.000",2.000
10,payroll_account,"26,521.000",3.000
11,emc_account,"24,733.000",4.000
9,pension_plan,"17,353.000",5.000
8,payroll,"16,333.000",6.000
5,long_term_deposit,"6,119.000",7.000
7,credit_card,"4,801.000",8.000
4,securities,"1,786.000",9.000
3,funds,"1,315.000",10.000


In [39]:

def get_top_n_recommendations(row, n=3):
    client_products = row[product_cols]
    not_owned_products = client_products[client_products == 0].index.tolist()
    ranking = product_ranking.set_index('Producto')['ranking']
    not_owned_ranking = ranking.loc[not_owned_products]
    if not not_owned_ranking.empty:
        # Obtener los top N productos con el ranking más alto
        recommended_products = not_owned_ranking.nsmallest(n).index.tolist()
        return recommended_products
    else:
        return None  # El cliente tiene todos los productos

# Aplicar la función para obtener las top N recomendaciones
df_last_partition['recomendaciones'] = df_last_partition.apply(get_top_n_recommendations, axis=1, n=3)

# Crear el DataFrame final con las recomendaciones
df_recommendations = df_last_partition[['pk_cid', 'recomendaciones']]

# Mostrar el DataFrame de recomendaciones
print(df_recommendations)

          pk_cid                             recomendaciones
5519929   657826  [debit_card, payroll_account, emc_account]
5519930   657817    [em_acount, debit_card, payroll_account]
5519931   657986  [em_acount, long_term_deposit, securities]
5519932   657905  [debit_card, payroll_account, emc_account]
5519933   657336  [debit_card, payroll_account, emc_account]
...          ...                                         ...
5962919  1166765  [debit_card, payroll_account, emc_account]
5962920  1166764  [debit_card, payroll_account, emc_account]
5962921  1166763  [debit_card, payroll_account, emc_account]
5962922  1166789  [debit_card, payroll_account, emc_account]
5962923  1550586  [debit_card, payroll_account, emc_account]

[442909 rows x 2 columns]
